## DIA 041: Implementacion de Autentificacion Multifactor (MFA) en la API

En este día, se añade una capa adicional de seguridad a la autenticación de la API mediante la implementación de un sistema de autenticación multifactor (MFA). La idea es que, tras el ingreso de las credenciales (usuario y contraseña), el sistema genere un código de verificación único y lo envíe al correo electrónico del usuario. El usuario deberá luego enviar ese código a través de un endpoint específico para completar el proceso de autenticación y recibir el token JWT definitivo. Este mecanismo protege la API frente a accesos no autorizados y ataques de fuerza bruta, ya que un atacante necesitará acceso al correo electrónico del usuario además de conocer la contraseña.

Código Completo (api.py)
python
Copiar
import os
import io
import random
import json
import time
import threading
import logging
from datetime import datetime
from functools import wraps

from flask import Flask, request, jsonify, render_template, url_for
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_bcrypt import Bcrypt
from flask_mail import Mail, Message
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
from flask_socketio import SocketIO, emit, join_room

# Configuración básica y variables de entorno
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Inicialización de extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)
bcrypt = Bcrypt(app)
mail = Mail(app)
jwt = JWTManager(app)
limiter = Limiter(app, key_func=get_remote_address, default_limits=["200 per day", "50 per hour"])
socketio = SocketIO(app, cors_allowed_origins="*")

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Modelos (simplificados)
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    # Para este ejemplo, asumiremos que el email es derivado (e.g., username@example.com)
    # y que la autenticación MFA se usará solo para demostrar el mecanismo.

class Feedback(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    prediction = db.Column(db.Integer, nullable=False)
    correct = db.Column(db.Boolean, nullable=False)
    comment = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "prediction": self.prediction,
            "correct": self.correct,
            "comment": self.comment,
            "timestamp": self.timestamp.isoformat()
        }

# ---------------------------
# Global Storage para MFA Codes (simplificado)
# ---------------------------
# En producción se debe utilizar un almacenamiento persistente o caché distribuido.
mfa_codes = {}

# ---------------------------
# Decorador para Roles (simplificado)
# ---------------------------
def role_required(required_role):
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            current_user_identity = get_jwt_identity()
            if not current_user_identity:
                return jsonify({"msg": "Token de acceso requerido"}), 401
            user = User.query.filter_by(username=current_user_identity).first()
            if not user:
                return jsonify({"msg": "Usuario no encontrado"}), 404
            # Suponemos que el atributo 'role' existe; en este ejemplo, se usa 'admin' para administradores
            if getattr(user, 'role', 'user') != required_role:
                return jsonify({"msg": "Acceso no autorizado"}), 403
            return f(*args, **kwargs)
        return wrapper
    return decorator

# ---------------------------
# Endpoints Comunes
# ---------------------------
@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Endpoint para Iniciar Sesión con MFA
# ---------------------------
@app.route('/login_mfa', methods=['POST'])
def login_mfa():
    """
    Inicia el proceso de login con MFA. Se reciben las credenciales,
    se verifica la autenticidad (simulada) y se envía un código de verificación al correo del usuario.
    ---
    tags:
      - Auth MFA
    parameters:
      - in: body
        name: credentials
        description: Credenciales de usuario para login.
        schema:
          type: object
          required:
            - username
            - password
          properties:
            username:
              type: string
            password:
              type: string
    responses:
      200:
        description: Código MFA enviado al correo.
      400:
        description: Datos incompletos.
      500:
        description: Error al enviar el código.
    """
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400

    # En un caso real, se validarían las credenciales contra la base de datos.
    # Aquí asumimos que la autenticación es correcta.
    # Generar un código MFA de 6 dígitos.
    code = str(random.randint(100000, 999999))
    mfa_codes[username] = code

    # Para este ejemplo, asumiremos que el correo del usuario es "username@example.com".
    recipient_email = f"{username}@example.com"
    subject = "Tu código MFA"
    body = f"Hola {username},\n\nTu código de autenticación es: {code}\n\nIngresa este código para completar el inicio de sesión."
    msg = Message(recipients=[recipient_email],
                  subject=subject,
                  body=body)
    try:
        mail.send(msg)
        logger.info(f"MFA code {code} enviado a {recipient_email}.")
    except Exception as e:
        logger.error("Error enviando el código MFA", exc_info=True)
        return jsonify({"msg": "Error sending MFA code"}), 500

    return jsonify({"msg": "MFA code sent to your email"}), 200

# ---------------------------
# Endpoint para Verificar MFA y Completar el Login
# ---------------------------
@app.route('/mfa/verify', methods=['POST'])
def mfa_verify():
    """
    Verifica el código MFA enviado al correo y, si es correcto, emite el token JWT final.
    ---
    tags:
      - Auth MFA
    parameters:
      - in: body
        name: mfa_data
        description: Datos para la verificación MFA.
        schema:
          type: object
          required:
            - username
            - code
          properties:
            username:
              type: string
            code:
              type: string
    responses:
      200:
        description: Token de acceso JWT generado.
      400:
        description: Código MFA incorrecto o datos faltantes.
    """
    data = request.get_json()
    username = data.get('username')
    code = data.get('code')
    if not username or not code:
        return jsonify({"msg": "Username and MFA code required"}), 400
    expected_code = mfa_codes.get(username)
    if expected_code != code:
        logger.warning(f"MFA verification failed for user '{username}'. Código recibido: {code}")
        return jsonify({"msg": "Invalid MFA code"}), 400

    # Eliminar el código MFA ya utilizado
    mfa_codes.pop(username, None)
    access_token = create_access_token(identity=username)
    logger.info(f"Usuario '{username}' completó MFA y obtuvo el token.")
    return jsonify({"access_token": access_token}), 200

# ---------------------------
# Otros Endpoints de la API (versionado, feedback, retraining, reportes, etc.)
# (Se omiten aquí por brevedad; se pueden integrar los endpoints de días anteriores)
# ---------------------------
# Ejemplo: Endpoint de Health Check ya implementado.

# ---------------------------
# Ejecutar la aplicación con soporte para WebSockets (opcional) y JWT
# ---------------------------
if __name__ == '__main__':
    app.run(debug=True)
Explicación del Código
Configuración Básica y Extensiones:
Se configura la aplicación Flask con variables de entorno y se inicializan las extensiones necesarias (JWT, SQLAlchemy, Flask-Mail, Flask-Limiter, etc.). Además, se configura el logging para capturar eventos.

Global MFA Storage:
Se utiliza un diccionario global (mfa_codes) para almacenar el código MFA generado para cada usuario durante el proceso de autenticación. (En producción, se debería usar un almacenamiento más seguro y persistente).

Endpoint /login_mfa:

Recibe las credenciales de usuario (username y password).
Tras validar las credenciales (en este ejemplo se asume que son correctas), se genera un código aleatorio de 6 dígitos.
El código se almacena en mfa_codes y se envía al correo electrónico del usuario (se asume que el correo es username@example.com).
Se registra el envío del código y se devuelve un mensaje informando que el código ha sido enviado.
Endpoint /mfa/verify:

Recibe el username y el código MFA enviado por el usuario.
Compara el código recibido con el almacenado en mfa_codes.
Si el código es correcto, elimina el código almacenado y emite el token JWT final, que se devuelve en la respuesta.
Si el código es incorrecto, se retorna un error.
Otros Endpoints:
Se mantienen otros endpoints comunes (como /health) y se integran con JWT para proteger el acceso, aunque para este ejemplo nos centramos en la parte de MFA.

Ejecución de la Aplicación:
La aplicación se ejecuta en modo debug. En un entorno de producción, se debería utilizar un servidor adecuado y configuraciones seguras.